# Data Analysis of Movies by Rating Relative to Age

In [9]:
import datetime as dt
import pandas as pd
import re

In [ ]:
# Data source: https://grouplens.org/datasets/movielens/20m/
# Successfully matched MD5 checksum with reference provided (data verified)

moviesMstr = pd.read_csv("/Users/Levient/Movie_Ratings/ml-20m/movies.csv")
ratingsMstr = pd.read_csv("/Users/Levient/Movie_Ratings/ml-20m/ratings.csv")

In [21]:
# All rows for 'movieId' and 'title(year)''
movies = moviesMstr[["movieId", "title"]]

# All rows for 'movieId' and 'rating'
ratings = ratingsMstr[["movieId", "rating"]]

print('Movies\n', movies.head(), '\nRatings\n', ratings.head())

Movies
    movieId                               title
0        1                    Toy Story (1995)
1        2                      Jumanji (1995)
2        3             Grumpier Old Men (1995)
3        4            Waiting to Exhale (1995)
4        5  Father of the Bride Part II (1995) 
Ratings
    movieId  rating
0        2     3.5
1       29     3.5
2       32     3.5
3       47     3.5
4       50     3.5


In [43]:
# Average all ratings per movie and add 'average column'
ratings.loc[:,'average'] = ratings.groupby('movieId')['rating'].transform('mean')

print('Average Ratings for All Movies\n', ratings.head())

Average Ratings for All Movies
    movieId  rating   average
0        2     3.5  3.211977
1       29     3.5  3.952230
2       32     3.5  3.898055
3       47     3.5  4.053493
4       50     3.5  4.334372


In [71]:
# Remove duplicate ratings rows from ratings, drop rating column

print('Before\n', ratings.shape)

average_ratings = ratings.drop_duplicates(subset=['movieId', 'average']).drop(columns=['rating'])

print('After\n', average_ratings.shape)
print('Total movies', movies.shape)
print('Averages\n', average_ratings.head())

# Issue to explore: There are more movies than there are average reviews. 

Before
 (20000263, 3)
After
 (26744, 2)
Total movies (27278, 3)
Averages
    movieId   average
0        2  3.211977
1       29  3.952230
2       32  3.898055
3       47  4.053493
4       50  4.334372


In [62]:
# Find median (and mean) of all ratings
ratings_median = ratings["average"].median()
ratings_mean = ratings["average"].mean()

print("Median of Average Ratings\n", ratings_median, "\nMean of Average Ratings\n", ratings_mean)

Median of Average Ratings
 3.601982097186701 
Mean of Average Ratings
 3.5255285642995635


In [47]:
# The classify_age function extracts a year in parens from 'title' and returns a string (e.g. 'old') 

def classify_age(row):
    year = extract_year(row)
    return assign_age(year)

def extract_year(row):
  year_in_parens = re.compile('(?<=\()(\d{4})(?=\))')
  result = year_in_parens.search(row['title'])
  if result:
    return result.group(0)
  else: 
    return None
  
def assign_age(year):
    if year == None:
      return "unknown"
    year = dt.datetime(int(year), 1, 1)
    if year < dt.datetime(1970, 1, 1):
        return "old"
    elif year < dt.datetime(1990, 1, 1):
        return "medium"
    else:
        return "new"

In [72]:
# Applying classify_age returns a new column 'age' with 4 classes: 'old', 'medium', 'new' and 'unknown'

movies['age'] = movies.apply(classify_age, axis=1)

print(movies.tail())

# If time permits: come back and print samples of each!!

       movieId                          title  age
27273   131254   Kein Bund für's Leben (2007)  new
27274   131256  Feuer, Eis & Dosenbier (2002)  new
27275   131258             The Pirates (2014)  new
27276   131260            Rentun Ruusu (2001)  new
27277   131262               Innocence (2014)  new


In [69]:
# Inner join movies and ratings by movieId. Validate one-to-one relationship.

movie_ratings = pd.merge(movies, average_ratings, validate="1:1")

print(movie_ratings.head())

   movieId                               title  age   average
0        1                    Toy Story (1995)  new  3.921240
1        2                      Jumanji (1995)  new  3.211977
2        3             Grumpier Old Men (1995)  new  3.151040
3        4            Waiting to Exhale (1995)  new  2.861393
4        5  Father of the Bride Part II (1995)  new  3.064592


In [98]:
# Sorting movies by average rating

movie_ratings = movie_ratings.sort_values(by=['average'], ascending=False)

print("Lowest by Rating\n", movie_ratings.tail())
print("\Highest by Rating\n", movie_ratings.head())

Lowest by Rating
        movieId                             title  age  average
22757   109359              Gerontophilia (2013)  new      0.5
23554   112778  Unrated II: Scary as Hell (2011)  new      0.5
15755    80154               Urban Menace (1999)  new      0.5
23553   112776                  Bloodline (2011)  new      0.5
24647   117630             Double Trouble (1992)  new      0.5
\Highest by Rating
        movieId                                          title  age  average
24643   117606                                 Divorce (1945)  old      5.0
23860   114011            New Rulers of the World, The (2001)  new      5.0
9005     26718  Life On A String (Bian chang Bian Zou) (1991)  new      5.0
26371   129352                                Freeheld (2007)  new      5.0
26416   129516                                  Poison (1951)  old      5.0


In [102]:
print('Length of Total Movie Ratings List\n', movie_ratings.shape)

# Top 20 percent
best = movie_ratings.iloc[:5349,:]
print('Best 20 percent\n', best.shape, '\n', best.head())

# Worst 20 percent
worst = movie_ratings.iloc[21395:,:]
print('\nWorst 20 percent\n', worst.shape, '\n', worst.tail())

# Middle 20 percent
ok = movie_ratings.iloc[10697:16047,:]
print('\nMiddle 20 percent\n', ok.shape, '\n', ok.tail())

Length of Total Movie Ratings List
 (26744, 4)
Best 20 percent
 (5349, 4) 
        movieId                                          title  age  average
24643   117606                                 Divorce (1945)  old      5.0
23860   114011            New Rulers of the World, The (2001)  new      5.0
9005     26718  Life On A String (Bian chang Bian Zou) (1991)  new      5.0
26371   129352                                Freeheld (2007)  new      5.0
26416   129516                                  Poison (1951)  old      5.0

Worst 20 percent
 (5349, 4) 
        movieId                             title  age  average
22757   109359              Gerontophilia (2013)  new      0.5
23554   112778  Unrated II: Scary as Hell (2011)  new      0.5
15755    80154               Urban Menace (1999)  new      0.5
23553   112776                  Bloodline (2011)  new      0.5
24647   117630             Double Trouble (1992)  new      0.5

Middle 20 percent
 (5350, 4) 
        movieId             